In [1]:
#Cargamos las librerías
import numpy as np
import bokeh.plotting as bkp
import bokeh.models as bkm
import bokeh.io as bkio
import bokeh.layouts as bkl
from bokeh.colors import RGB
import pandas as pd
import datetime as dt

#Mostramos las gráficas dentro del notebook
bkp.output_notebook()

Loading BokehJS ...

In [2]:
#Cargamos los datos https://www.nodc.noaa.gov/OC5/3M_HEAT_CONTENT/basin_tsl_data.html

niveltotal=pd.read_csv("GMSL_TPJAOS_4.2_199209_201803.txt", sep="\s+", index_col=None, header=None, comment="H", 
                        names=["altimeter type","merged file cycle", "year","number of observations",
                               "number of weighted observations", "GSML NOT GIA", "sd GSML NOT GIA", "GSML NOT GIA smoothed 60 day", "GSML", 
                               "sd GSML", "GSML smoothed 60 day", "GSML smoothed 60 day and annual cycles"])
niveltmp=pd.read_csv("pent_a-mm-w0-2000m.dat.txt", sep="\s+", header=0, index_col=None)

In [3]:
niveltotal.head(20)

altimeter type  merged file cycle         year  number of observations  \
0                0                 11  1993.011526                  466462   
1                0                 12  1993.038692                  460889   
2                0                 13  1993.065858                  472123   
3                0                 14  1993.093025                  421377   
4                0                 15  1993.120191                  459548   
5                0                 16  1993.147357                  416480   
6                0                 17  1993.174523                  467980   
7                0                 18  1993.201689                  466075   
8                0                 19  1993.228855                  461207   
9              999                 20  1993.256021                  467274   
10               0                 21  1993.283188                  461743   
11               0                 22  1993.310354                  469833   
12               0                 23  1993.337520                  467659   
13               0                 24  1993.364686                  467923   
14               0                 25  1993.391852                  470038   
15               0                 26  1993.419018                  471715   
16               0                 27  1993.446185                  468842   
17               0                 28  1993.473351                  468765   
18               0                 29  1993.500517                  448241   
19               0                 30  1993.527683                  461232   

    number of weighted observations  GSML NOT GIA  sd GSML NOT GIA  \
0                         337277.00        -37.24            92.66   
1                         334037.31        -40.35            95.39   
2                         342416.09        -40.17            92.29   
3                         306050.59        -41.92            96.20   
4                         331094.09        -36.89            94.64   
5                         300344.81        -35.69            93.22   
6                         337061.91        -36.09            91.67   
7                         335414.41        -34.70            91.95   
8                         331521.09        -34.04            91.70   
9                         340098.59        -37.95           101.31   
10                        331709.69        -36.48            89.84   
11                        338035.69        -33.09            89.52   
12                        337536.50        -35.11            88.88   
13                        336246.41        -37.76            88.73   
14                        337664.19        -39.00            89.03   
15                        339521.19        -38.36            88.65   
16                        338293.00        -35.35            89.34   
17                        339163.81        -35.44            88.12   
18                        326745.50        -36.07            86.35   
19                        336487.41        -37.01            88.23   

    GSML NOT GIA smoothed 60 day   GSML  sd GSML  GSML smoothed 60 day  \
0                         -37.02 -37.24    92.66                -37.02   
1                         -38.20 -40.34    95.39                -38.19   
2                         -38.28 -40.16    92.29                -38.27   
3                         -38.56 -41.89    96.19                -38.54   
4                         -37.87 -36.85    94.63                -37.83   
5                         -36.96 -35.65    93.21                -36.93   
6                         -36.14 -36.05    91.67                -36.09   
7                         -35.53 -34.65    91.94                -35.48   
8                         -35.18 -33.98    91.68                -35.12   
9                         -35.09 -37.88   101.29                -35.02   
10                        -35.23 -36.41    89.83                -35.16   
11    

In [4]:
#Leemos las primeras 5 filas para ver que se han cargado bien los datos
(niveltmp[(niveltmp["YEAR"]>1993)]).head(5)

YEAR     WO   WOse      NH   NHse     SH   SHse
36  1993.5  6.511  0.560   7.745  0.434  5.646  0.649
37  1994.5  5.651  0.560   9.669  0.469  2.832  0.624
38  1995.5  6.397  0.580   9.454  0.479  4.251  0.651
39  1996.5  7.255  0.659  11.640  0.569  4.178  0.722
40  1997.5  7.152  0.736  11.858  0.589  3.850  0.840

In [5]:
#Fijamos el 0 de la variación del nivel del mar al 1 de Julio de 1993
niveltmp["variacion"]=niveltmp["WO"]-niveltmp.at[36,"WO"]
niveltotal["variacion"]=niveltotal["GSML smoothed 60 day and annual cycles"]-niveltotal.at[18,"GSML smoothed 60 day and annual cycles"]

In [6]:
#Cargamos los colores elegidos como paleta para este MOOC
p_azulmarino = RGB(0, 28, 100)
p_azulclaro = RGB(0, 131, 255)
p_amarillo = RGB(244, 255, 84)
p_verde = RGB(172, 255, 165)
p_rojo = RGB(255, 67, 67)

In [7]:
#Creamos una figura de Bokeh que va a contener nuestra gráfica. Indico el tamaño, que el eje X van a ser fechas
#y las herramientas activas (mover la gráfica y hacer zoom con el ratón)

p = bkp.figure(tools=["pan","wheel_zoom","box_zoom","save,reset"], width=900, height=500, active_drag = "pan", active_scroll = "wheel_zoom")


#Represento la subida del nivel del mar por motivos térmicos como una línea azul
p2=p.line(source=niveltmp, x='YEAR',y='variacion', color=p_azulmarino, alpha=0.7,line_width=3, legend="Variación Térmica del Nivel del Mar (promedio de 5 años)")
p.add_tools(bkm.HoverTool(renderers=[p2], tooltips=[("Variación Térmica", "@{variacion}{0,0.0} mm (@{YEAR}{0,0})")],mode='vline'))

#Represento la suibdad total del nivel del mar como una linea rojo
p3=p.line(source=niveltotal, x='year',y='variacion', color=p_rojo, alpha=1,line_width=3, legend="Variación Total del Nivel del Mar")
p.add_tools(bkm.HoverTool(renderers=[p3], tooltips=[("Nivel del Mar", "@{variacion}{0,0.0} mm (@{year}{0,0})")],mode='vline'))

In [8]:
#Indico las propiedades del gráfico
p.title.text = "Variación en el nivel del mar"
p.title.text_color = "black"
p.title.text_font = "ADAM CG PRO"
p.title.text_font_style = "normal"
p.title.text_font_size = "16pt"
p.title.align = "center"

p.background_fill_color = "white"
p.background_fill_alpha = 1.

p.xaxis.axis_label = 'Año'
p.xaxis.axis_label_text_font = "ADAM CG PRO"
p.xaxis.axis_label_text_font_style  = "normal"
p.xaxis.axis_label_text_font_size = "12pt"
p.yaxis.axis_label = 'mm'
p.yaxis.axis_label_text_font = "ADAM CG PRO"
p.yaxis.axis_label_text_font_style  = "normal"
p.yaxis.axis_label_text_font_size = "12pt"

p.yaxis[0].formatter = bkm.NumeralTickFormatter(format="0,0")

p.x_range = bkm.Range1d(start=1993, end=2018)

p.ygrid.grid_line_color=p_azulclaro
p.ygrid.grid_line_alpha=0.25
p.xgrid.grid_line_color=p_azulclaro
p.xgrid.grid_line_alpha=0.0

p.xaxis.major_tick_line_color = p_rojo
p.xaxis.major_tick_line_width = 3
p.xaxis.minor_tick_line_color = p_rojo
p.xaxis.minor_tick_line_width = 1
p.yaxis.major_tick_line_color = p_rojo
p.yaxis.major_tick_line_width = 3
p.yaxis.minor_tick_line_color = p_rojo
p.yaxis.minor_tick_line_width = 1

p.outline_line_width = 7
p.outline_line_alpha = 0.5
p.outline_line_color = p_azulclaro

p.legend.location = "top_left"
p.legend.click_policy="hide"
p.legend.label_text_font = "ADAM CG PRO"

In [9]:
#Si deseamos guardar la gráfica como un fichero html, descomentar la siguiente línea de código,
#al ejecutarse el notebook generará un html en el directorio
#p.output_backend = "canvas"
#bkp.output_file("NivelMar.html", title='Bokeh Plot', mode='cdn', root_dir=None)

#para guardar un svg usando el menú de la gráfica (en lugar de un PNG), descomentar esta línea
p.output_backend = "svg"

bkp.show(p)